# Demonstrate Seq2Seq Wrapper with twitter chat log

In [2]:
!pip install tensorflow

    100% |████████████████████████████████| 46.5MB 38kB/s eta 0:00:011    41% |█████████████▏                  | 19.1MB 2.5MB/s eta 0:00:11    90% |████████████████████████████▉   | 41.9MB 6.7MB/s eta 0:00:01
    100% |████████████████████████████████| 1.6MB 1.1MB/s eta 0:00:01
    100% |████████████████████████████████| 3.1MB 521kB/s ta 0:00:011
    100% |████████████████████████████████| 1.2MB 1.3MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 4.7MB/s ta 0:00:011
    100% |████████████████████████████████| 890kB 1.4MB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 2.3MB/s ta 0:00:01
  Running setup.py bdist_wheel for termcolor ... done
  Stored in directory: /Users/yanasavva/Library/Caches/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Running setup.py bdist_wheel for gast ... done
  Stored in directory: /Users/yanasavva/Library/Caches/pip/wheels/9a/1f/0e/3cde98113222b853e98fc0a8e9924480a3e25f1b4008cedb4f
  Running setup.py 

In [1]:
import tensorflow as tf
import numpy as np

# preprocessed data
from datasets.twitter import data
import data_utils

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='datasets/twitter/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [3]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024

In [4]:
import seq2seq_wrapper

In [5]:
import importlib
importlib.reload(seq2seq_wrapper)

<module 'seq2seq_wrapper' from '/Users/yanasavva/practical_seq2seq/seq2seq_wrapper.py'>

In [6]:
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/twitter/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Building Graph </log>

In [8]:
tf.placeholder

<function tensorflow.python.ops.array_ops.placeholder>

In [9]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 256)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [9]:
sess = model.train(train_batch_gen, val_batch_gen)


<log> Training started </log>

Model saved to disk at iteration #500
val   loss : 3.370399
Interrupted by user at iteration 565


In [7]:
sess = model.restore_last_session()

INFO:tensorflow:Restoring parameters from ckpt/twitter/seq2seq_model.ckpt-1000


In [10]:
input_ = test_batch_gen.__next__()[0]
output = model.predict(sess, input_)
print(output.shape)

(256, 20)


In [17]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
#     if decoded.count('unk') == 0:
    if decoded not in replies:
        print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
        replies.append(decoded)

q : [подскажи]; a : [я unk unk]
q : [супер  тогда unk в среду в 6 на павелецкой]; a : [а unk unk]
q : [и я вот все никак не могу понять что там unk с этими unk unk на unk кудато идут постоянно]; a : [unk]


In [14]:
input_.T

array([[2697,    0,    0, ...,    0,    0,    0],
       [  11,   54,   22, ...,    0,    0,    0],
       [  92,  994,    0, ...,    0,    0,    0],
       ...,
       [ 665,    0,    0, ...,    0,    0,    0],
       [   4,    0,    0, ...,    0,    0,    0],
       [ 383,    0,    0, ...,    0,    0,    0]], dtype=int32)

In [15]:
data_utils.decode(sequence=input_.T[0], lookup=metadata['idx2w'], separator=' ')

'подскажи'

In [16]:
data_utils.decode(sequence=output[0], lookup=metadata['idx2w'], separator=' ').split(' ')

['я', 'unk', 'unk']